# EEG preprocessing 

In this notebook: 
- Necessary imports
- Data loader for events, eeg and meta data
- Filtering algorithm
- EEG raw to epochs
- saving filtered data to `metadata.csv`

Preprocessing steps: 
+ Prepare EEG (1. Subtract reference (mastoids), 2. Detrend, 3. Filter, 4. Remove bad channels)
+ Segment EEG into standard and deviant epochs (ERPs) (1. subtract baseline, 2. Reject artefacts, 3. Average (for each marker/subject/channel separately))
+ Calculate Mismatch response (deviant - standard for a single subject) (check differences between channels and subjects)

## Imports

The data will be processed using the mne library. Also there are libraries made in eegyolk in order to load the metadata, eeg data and the event markers. Those libraries need to be imported

In [3]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import json

import eegyolk
import eegyolk.helper_functions as hf # library useful for eeg and erp data cleaning
import eegyolk.initialization_functions #library to import data
import eegyolk.epod_helper


## Load metadata and eeg files

First the different pathways for the different datasets need to be defined. There are three pathways: eeg, metadata and events. The files can be loaded using the initialization_functions library. All event markers needs to be saved in a seperate folder. If not saved already, the event markers will be saved using the initialization_function library. The data must be saved in a separate folder called "epod_data_not_pushed" in the ePodium repository. 

In [4]:
with open("config.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

data_path = jsonObject['root']
path_eeg = jsonObject['dataset']
path_metadata = jsonObject['metadata']
path_eventmarkers = jsonObject['events']

print(path_eeg)
print(path_metadata)
print(path_eventmarkers)

F:/Stage/ePODIUM/Data/ePodium_projectfolder/Dataset
F:/Stage/ePODIUM/Data/ePodium_projectfolder/Metadata
F:/Stage/ePODIUM/Data/ePodium_projectfolder/events


In [5]:
# load metadata
files_metadata = ["children.txt", "cdi.txt", "parents.txt", "CODES_overview.txt"]  
children, cdi, parents, codes = eegyolk.initialization_functions.i_load_metadata(path_metadata, files_metadata)

In [6]:
# load eeg
eeg, eeg_filename =  eegyolk.initialization_functions.load_dataset(path_eeg, preload=False) # preload must be set to True once on the cloud

Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\101a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
1 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\101b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
2 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\102a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
3 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\102b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
4 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\103a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
5 EEG files loaded
Extracting EDF 

C:\Users\nadin\anaconda3\lib\site-packages\eegyolk\initialization_functions.py:55: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw = mne.io.read_raw_bdf(path, preload=preload)
C:\Users\nadin\anaconda3\lib\site-packages\eegyolk\initialization_functions.py:55: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw = mne.io.read_raw_bdf(path, preload=preload)


Creating raw.info structure...
36 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\119a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
37 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\119b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
38 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\120a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
39 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\120b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
40 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\121a.bdf...
BDF file detected
Setting channel info structure...
Creating r

BDF file detected
Setting channel info structure...
Creating raw.info structure...
78 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\140b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
79 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\141a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
80 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\142a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
81 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\142b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
82 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\143a.bdf...
BDF file d

Creating raw.info structure...
119 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\162a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
120 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\162b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
121 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\163a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
122 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\163b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
123 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\164a.bdf...
BDF file detected
Setting channel info structure...
Creat

160 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\179a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
161 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\179b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
162 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\180a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
163 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\180b.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
164 EEG files loaded
Extracting EDF parameters from F:\Stage\ePODIUM\Data\ePodium_projectfolder\Dataset\181a.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
165 E

C:\Users\nadin\anaconda3\lib\site-packages\eegyolk\initialization_functions.py:55: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw = mne.io.read_raw_bdf(path, preload=preload)


In [7]:
# load events 
events_files = os.listdir(path_eventmarkers)
if len(events_files) == 0 or path_eventmarkers == False: # check if event markers are saved in a seperate folder
    eegyolk.initialization_functions.save_event_markers(path_eventmarkers, eeg, eeg_filename) # save event markers

event_markers = eegyolk.initialization_functions.load_events(path_eventmarkers, eeg_filename) # load event markers
event_markers_simplified = eegyolk.epod_helper.group_events_12(event_markers) # simplify events

194 Event Marker files loaded


## Filtering raw EEG 

### Set filter parameters

Below you can define the frequencies for the bandpass filter. The lowpass can not be below 0 and the highpass can not be higher then 100. Most common bandpass filter is filtering between 0.1 and 30. 

In [8]:
lowpass = widgets.BoundedFloatText(
    value=0.1,
    min=0,
    max=100,
    step=0.1,
    description='lowpass:',
    disabled=False
)

highpass = widgets.BoundedFloatText(
    value=40,
    min=0,
    max=100,
    step=0.1,
    description='highpass:',
    disabled=False
)

widgets.VBox([lowpass,highpass])


In [9]:
# change type to integer
lowpass = float(lowpass.value)
highpass = float(highpass.value)

The number of freqs can vary and be adjusted by changing `n`. The used frequency for this analysis is `[60, 120, 180, 240]`.

In [10]:
n = 4
freq = list(widgets.BoundedIntText(
    description='freq[{}]'.format(i),
    min=0,
    max=300,
    step=1,
    value=(i+1)*60)
    for i in range(n))

widgets.VBox(children=freq)

In [11]:
freqs= [f.value for f in freq]

Epochs are created with joining the eeg data with a specific event.  mne.Epochs automaticaly create a baseline correction and artefact rejection. 

In [12]:
event_dictionary = eegyolk.epod_helper.event_dictionary
event_dictionary

{'GiepMT_FS': 1,
 'GiepMT_S': 2,
 'GiepMT_D': 3,
 'GiepST_FS': 4,
 'GiepST_S': 5,
 'GiepST_D': 6,
 'GopMT_FS': 7,
 'GopMT_S': 8,
 'GopMT_D': 9,
 'GopST_FS': 10,
 'GopST_S': 11,
 'GopST_D': 12}

In order to create the epochs, the time before `tmin` and after an event `tmax` needs to be defined. The default values are set to -0.2 and 0.8. `tmin` and `tmax` are the start and stop time relative to each event.

In [13]:
tmin = widgets.BoundedFloatText(
    value=-0.2,
    min=-1,
    max=1,
    step=0.1,
    description='tmin:',
    disabled=False
)

tmax = widgets.BoundedFloatText(
    value=0.8,
    min=-1,
    max=1,
    step=0.1,
    description='tmax:',
    disabled=False
)

widgets.VBox([tmin,tmax])

In [14]:
tmin = float(tmin.value)
tmax = float(tmax.value)

### Filter generator

This filter contains a bandpass filter, with as input the parameters `lowpass` and `highpass`. It also contains a notch filter to filter out power line noise and needs as input `freqs` for frequencies to apply the notch filter on. The next input is `mastoid_channels`, to subtract the reference from the raw eeg data. Finally, channels from the eeg can be dropped by adjusting the `drop_ch`. 

For a selected event, an interval is created with a time before and after event. This represents an epoch. The function automatically performs a baseline correction. 

In [15]:
mastoid_channels = ['EXG1', 'EXG2']
drop_ch = ['EXG1', 'EXG2','EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status']

def filter_raweeg_gen(eeg, lowpass, highpass, freqs, mastoid_channels, drop_ch):
    for i in range(len(eeg)): 
        processed_file = os.path.join(epoch_folder, eeg_filename[i]+"_epo.fif")
        if not os.path.exists(processed_file):
            yield hf.filter_eeg_raw(eeg[i].load_data(), lowpass, highpass, freqs, mastoid_channels, drop_ch)
        else: 
            yield print(f"File {processed_file} already processed \n", end = '')
            

In [16]:
filtered_eegs = filter_raweeg_gen(eeg, lowpass, highpass, freqs, mastoid_channels, drop_ch)
epoch_folder = os.path.join(data_path, "epochs")

if not os.path.exists(epoch_folder):
        os.mkdir(epoch_folder)
        
for idx, single_eeg in enumerate(filtered_eegs):
    processed_file = os.path.join(epoch_folder, eeg_filename[idx]+"_epo.fif")
    if not os.path.exists(processed_file):
        epoch = hf.create_epoch(single_eeg, event_markers_simplified[idx], tmin, tmax)
        epoch_file = os.path.join(epoch_folder, eeg_filename[idx]+"_epo.fif")
        epoch.save(epoch_file, overwrite=True)
        print("\n", idx+1, " saved.")


File F:/Stage/ePODIUM/Data/ePodium_projectfolder\epochs\101a_epo.fif already processed 
File F:/Stage/ePODIUM/Data/ePodium_projectfolder\epochs\101b_epo.fif already processed 
File F:/Stage/ePODIUM/Data/ePodium_projectfolder\epochs\102a_epo.fif already processed 
File F:/Stage/ePODIUM/Data/ePodium_projectfolder\epochs\102b_epo.fif already processed 
File F:/Stage/ePODIUM/Data/ePodium_projectfolder\epochs\103a_epo.fif already processed 
File F:/Stage/ePODIUM/Data/ePodium_projectfolder\epochs\103b_epo.fif already processed 
File F:/Stage/ePODIUM/Data/ePodium_projectfolder\epochs\104a_epo.fif already processed 
File F:/Stage/ePODIUM/Data/ePodium_projectfolder\epochs\104b_epo.fif already processed 
File F:/Stage/ePODIUM/Data/ePodium_projectfolder\epochs\105a_epo.fif already processed 
File F:/Stage/ePODIUM/Data/ePodium_projectfolder\epochs\105b_epo.fif already processed 
File F:/Stage/ePODIUM/Data/ePodium_projectfolder\epochs\106a_epo.fif already processed 
File F:/Stage/ePODIUM/Data/ePodi

## Create DataFrame with metadata and eeg/epoch paths

In [17]:
parents.rename(columns = {'child':'ParticipantID'}, inplace=True)
cdi.rename(columns = {'participant':'ParticipantID'}, inplace=True)

In [18]:
metadata = pd.merge(cdi, children, on="ParticipantID")
metadata = pd.merge(metadata, parents, on="ParticipantID")

In [19]:
metadata.columns

Index(['ParticipantID', 'test', 'sex', 'age_months', 'age_months_days',
       'dyslexic_parent', 'b01 - Geluidseffecten en dierengeluiden',
       'b02 - Dierennamen', 'b03 - Voertuigen', 'b04 - Speelgoed',
       'b05 - Eten en drinken', 'b06 - Kleding', 'b07 - Delen van het lichaam',
       'b08 - Kleine huishoudelijke voorwerpen', 'b09 - Meubels en kamers',
       'b10 - Voorwerpen buitenshuis', 'b11 - Plaatsen buitenshuis',
       'b12 - Mensen', 'b13 - Spelletjes en routines',
       'b14 - Omschrijvende woorden', 'b15 - Werkwoorden',
       'b16 - Woorden over tijd', 'b17 - Voornaamwoorden',
       'b18 - Vragende woordjes', 'b19 - Voorzetsels en plaatsbepalingen',
       'b20 - Hoeveelheden en lidwoorden', 'b21 - Hulpwerkwoorden',
       'b22 - Voegwoorden', 'p01 - Geluidseffecten en dierengeluiden',
       'p02 - Dierennamen', 'p03 - Voertuigen', 'p04 - Speelgoed',
       'p05 - Eten en drinken', 'p06 - Kleding', 'p07 - Delen van het lichaam',
       'p08 - Kleine huishoudelij

In [20]:
metadata = metadata.drop(['age_months_days','b01 - Geluidseffecten en dierengeluiden',
       'b02 - Dierennamen', 'b03 - Voertuigen', 'b04 - Speelgoed',
       'b05 - Eten en drinken', 'b06 - Kleding', 'b07 - Delen van het lichaam',
       'b08 - Kleine huishoudelijke voorwerpen', 'b09 - Meubels en kamers',
       'b10 - Voorwerpen buitenshuis', 'b11 - Plaatsen buitenshuis',
       'b12 - Mensen', 'b13 - Spelletjes en routines',
       'b14 - Omschrijvende woorden', 'b15 - Werkwoorden',
       'b16 - Woorden over tijd', 'b17 - Voornaamwoorden',
       'b18 - Vragende woordjes', 'b19 - Voorzetsels en plaatsbepalingen',
       'b20 - Hoeveelheden en lidwoorden', 'b21 - Hulpwerkwoorden',
       'b22 - Voegwoorden', 'p01 - Geluidseffecten en dierengeluiden',
       'p02 - Dierennamen', 'p03 - Voertuigen', 'p04 - Speelgoed',
       'p05 - Eten en drinken', 'p06 - Kleding', 'p07 - Delen van het lichaam',
       'p08 - Kleine huishoudelijke voorwerpen', 'p09 - Meubels en kamers',
       'p10 - Voorwerpen buitenshuis', 'p11 - Plaatsen buitenshuis',
       'p12 - Mensen', 'p13 - Spelletjes en routines',
       'p14 - Omschrijvende woorden', 'p15 - Werkwoorden',
       'p16 - Woorden over tijd', 'p17 - Voornaamwoorden',
       'p18 - Vragende woordjes', 'p19 - Voorzetsels en plaatsbepalingen',
       'p20 - Hoeveelheden en lidwoorden', 'p21 - Hulpwerkwoorden',
       'p22 - Voegwoorden', 'wB - Woorduiteinden', 'wC - Woordvormen',
       'zE - Zinnen',  'Woordvormen - Ruwe score', 'Woordvormen - Percentiel',
       'Woordvormen - Taalleeftijd', 'Zinnen - Ruwe score',
       'Zinnen - Percentiel', 'Zinnen - Taalleeftijd', 'Woordenschatbegrip - Ruwe score',
       'Woordenschatbegrip - Percentiel', 'Woordenschatbegrip - Taalleeftijd',
       'Woordenschatproductie - Ruwe score',
       'Woordenschatproductie - Percentiel',
       'Woordenschatproductie - Taalleeftijd', 'CDIpresent_a', 'CDIpresent_b',  'Sex', 'Age_original_a', 'Age_days_a', 'Age_months_a',
       'Age_original_b', 'Age_days_b', 'Age_months_b', 'emt_mother','klepel_mother', 'vc_mother', 'dyslexia_mother_accToMother',
       'emt_father', 'klepel_father', 'vc_father',
       'dyslexia_father_accToFather'], axis = 1)

In [21]:
metadata['eeg_file']= metadata['ParticipantID'].astype(str) + metadata['test']

In [42]:
epoch_folder = os.path.join(data_path, "epochs_clean").replace("\\","/") 

In [43]:
epoch_filename = []

# Iterate directory
for path in os.listdir(epoch_folder):
    # check if current path is a file
    if os.path.isfile(os.path.join(epoch_folder, path)):
        epoch_filename.append(path)

In [44]:
df_eegfilenames = pd.DataFrame(eeg_filename, columns=['eeg_file'])
df_epochfilenames = pd.DataFrame(epoch_filename, columns=['epoch_file'])

In [45]:
df_epochfilenames['eeg_file'] = df_epochfilenames.epoch_file.str[:4]

In [46]:
metadata['path_eeg'] = path_eeg
metadata['path_epoch'] = epoch_folder
metadata['path_eventmarkers'] = path_eventmarkers

In [47]:
metadata['path_epoch']

0      F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...
1      F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...
2      F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...
3      F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...
4      F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...
                             ...                        
217    F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...
218    F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...
219    F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...
220    F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...
221    F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...
Name: path_epoch, Length: 222, dtype: object

In [48]:
df = pd.merge(df_eegfilenames, metadata, on='eeg_file')
df = pd.merge(df, df_epochfilenames, on='eeg_file')

In [49]:
drop_files = ["113a", "107b (deel 1+2)", "132a", "121b(2)", "113b", "107b (deel 3+4)", "147a",
                "121a", "134a", "143b", "121b(1)", "145b", "152a", "184a", "165a", "151a", "163a", "179a","179b", "182b", "186a", "193b"]

df = df[~df['eeg_file'].isin(drop_files)]
df = df.drop(df[df['test'] == "b"].index).reset_index(drop=True)

In [50]:
df.to_csv('metadata.csv', index=False)

In [51]:
df

,eeg_file,ParticipantID,test,sex,age_months,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,101a,101,a,m,20,m,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,101a_epo.fif
1,102a,102,a,f,20,Nee,Control,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,102a_epo.fif
2,103a,103,a,f,20,m,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,103a_epo.fif
3,104a,104,a,m,18,f,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,104a_epo.fif
4,105a,105,a,f,17,f,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,105a_epo.fif
5,106a,106,a,m,19,f,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,106a_epo.fif
6,107a,107,a,f,16,m,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,107a_epo.fif
7,109a,109,a,m,21,m,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,109a_epo.fif
8,110a,110,a,m,17,m,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,110a_epo.fif
9,111a,111,a,m,20,m,At risk,F:/Stage/ePODIUM/Data/ePodium_projectfolder/Da...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ep...,F:/Stage/ePODIUM/Data/ePodium_projectfolder/ev...,111a_epo.fif
